# Classifying hand gestures with LSTM and Keras

In [28]:
# Gather data set into paths and class labels
import glob, os
import re

paths = []
labels = []
for path, subdirs, files in os.walk('DigitData'):
    for name in files:
        paths.append(os.path.join(path, name))
        labels.append(re.split(r'(\d+)', name)[0])
print(len(paths), len(labels))

315 315


In [29]:
# Verify we have 15 unique classes
myset = set(labels)
num_classes = len(myset)
print(myset, num_classes)
import pandas as pd
#Get shape if input data
df = pd.read_csv(paths[0], header=None, nrows=20000)
timestep = df.shape[0]
num_channels = df.shape[1]

{'M_R', 'IMR', 'M_M', 'T_R', 'I_M', 'T_I', 'HC_', 'T_T', 'T_M', 'MRL', 'I_I', 'L_L', 'R_L', 'T_L', 'R_R'} 15


In [30]:
#Dealing with x data
import numpy as np
#Initialize x data
x_data = np.empty((len(paths), timestep, num_channels), dtype=np.float32)
#Load dat to tensor
for i, path in enumerate(paths):
    data = pd.read_csv(path, header=None, nrows=20000)
    x_data[i,:,:] = data.values
import scipy.signal as sg
decimated = sg.decimate(x_data, axis = 1, q =20)

In [31]:
#Dealing with y data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#Change class label to one-hot vectors 
le = LabelEncoder()
one_hot = OneHotEncoder(sparse = False)

#Convert to ints
le.fit(list(myset))
int_labels = le.transform(labels)
int_labels

array([ 5, 10,  4, 14, 11,  0,  6, 13,  1,  9, 14, 12, 11,  7,  9,  6,  7,
        8,  2, 12, 11,  1,  5,  8,  8,  1,  3, 14,  4,  5,  6,  2,  0,  4,
       13,  0,  2, 10, 12,  9, 13, 10,  3,  7,  3,  5, 10,  4, 14, 11,  0,
        6, 13,  1,  9, 14, 12, 11,  7,  9,  6,  7,  8,  2, 12, 11,  1,  5,
        8,  8,  1,  3, 14,  4,  5,  6,  2,  0,  4, 13,  0,  2, 10, 12,  9,
       13, 10,  3,  7,  3,  5, 10,  4, 14, 11,  0,  6, 13,  1,  9, 14, 12,
       11,  7,  9,  6,  7,  8,  2, 12, 11,  1,  5,  8,  8,  1,  3, 14,  4,
        5,  6,  2,  0,  4, 13,  0,  2, 10, 12,  9, 13, 10,  3,  7,  3,  5,
       10,  4, 14, 11,  0,  6, 13,  1,  9, 14, 12, 11,  7,  9,  6,  7,  8,
        2, 12, 11,  1,  5,  8,  8,  1,  3, 14,  4,  5,  6,  2,  0,  4, 13,
        0,  2, 10, 12,  9, 13, 10,  3,  7,  3,  5, 10,  4, 14, 11,  0,  6,
       13,  1,  9, 14, 12, 11,  7,  9,  6,  7,  8,  2, 12, 11,  1,  5,  8,
        8,  1,  3, 14,  4,  5,  6,  2,  0,  4, 13,  0,  2, 10, 12,  9, 13,
       10,  3,  7,  3,  5

In [32]:
#Splitting data to train test val split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(decimated,int_labels, test_size = 0.2, random_state=40, 
                                                   stratify = int_labels)

In [33]:
#Normalize the data
channel_means = np.mean(X_train, axis = (0,1)).reshape(1,8)
channel_std = np.std(X_train, axis = (0,1)).reshape(1,8)
X_train -= channel_means
X_train /= channel_std

In [34]:
#Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state=40, 
                                                 stratify = y_train)

In [35]:
#Set up Keras model
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(256,
              input_shape = (1000, num_channels), return_sequences = True))
model.add(LSTM(256, return_sequences = True))
model.add(LSTM(256))
model.add(Dense(num_classes, activation = 'softmax'))

In [36]:
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer= 'adam',
             metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_val,y_val))

Train on 201 samples, validate on 51 samples
Epoch 1/50
201/201 [==============================] - 74s 366ms/step - loss: 2.6994 - acc: 0.0846 - val_loss: 2.5872 - val_acc: 0.0980
Epoch 2/50
201/201 [==============================] - 70s 349ms/step - loss: 2.5523 - acc: 0.1741 - val_loss: 2.4707 - val_acc: 0.1569
Epoch 3/50
201/201 [==============================] - 70s 349ms/step - loss: 2.4121 - acc: 0.1542 - val_loss: 2.4761 - val_acc: 0.1176
Epoch 4/50
201/201 [==============================] - 71s 354ms/step - loss: 2.4256 - acc: 0.1393 - val_loss: 2.5141 - val_acc: 0.1373
Epoch 5/50
201/201 [==============================] - 73s 362ms/step - loss: 2.3919 - acc: 0.1692 - val_loss: 2.4682 - val_acc: 0.1765
Epoch 6/50
201/201 [==============================] - 72s 357ms/step - loss: 2.3132 - acc: 0.2040 - val_loss: 2.3155 - val_acc: 0.1765
Epoch 7/50
201/201 [==============================] - 72s 358ms/step - loss: 2.2711 - acc: 0.1542 - val_loss: 2.3645 - val_acc: 0.1373
Epoch 8/50